In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
seed = 42

df = pd.read_csv("D:/imdb/final_IMDB_WIKI.csv")
train_df, val_df = train_test_split(df, test_size=0.2, random_state=seed)

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="D:/imdb/",
    x_col="filename",
    y_col="age",
    batch_size=64,
    target_size=(224, 224),
    class_mode="raw",
    shuffle=True,
    seed=seed
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory="D:/imdb/",
    x_col="filename",
    y_col="age",
    batch_size=64,
    target_size=(224, 224),
    class_mode="raw",
    shuffle=True,
    seed=seed
)

Found 55509 validated image filenames.
Found 13878 validated image filenames.


In [6]:
from keras.applications.mobilenet import MobileNet
from keras.applications import ResNet50
from keras.models import Model, Sequential
import coral_ordinal as coral
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications import MobileNetV3Small 
from keras.losses import CategoricalCrossentropy

loss_array = {}
metrics_array = {}
base_model = MobileNet(input_shape =  (224,224, 3), 
                                 include_top = False, 
                                 weights = 'imagenet', pooling='avg')

outputs = coral.CoralOrdinal(101)(base_model.output)
model = Model(inputs=base_model.input, outputs=outputs)

for layer in model.layers[:10]:
    layer.trainable=False
for layer in model.layers[10:]:
    layer.trainable=True

print(model.summary())
model.compile(optimizer='adam', loss=coral.OrdinalCrossEntropy(num_classes=101), metrics=[coral.MeanAbsoluteErrorLabels()])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [4]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

early_stopping = EarlyStopping(
                                patience=10, 
                                min_delta = 0.01,      
                                verbose=1, 
                                mode = 'min',
                                monitor='val_loss')

reduce_learning_rate = ReduceLROnPlateau(
                                    monitor="val_loss",
                                    patience=5,
                                    episilon= 0.01,
                                    factor=0.1,
                                    cooldown = 0, 
                                    verbose=1)


In [7]:
score = model.fit(train_generator, epochs=200, validation_data=val_generator,callbacks=[
    reduce_learning_rate,
    early_stopping
] ,verbose=True)

Epoch 1/100
868/868 [==============================] - 286s 326ms/step - loss: 55.3469 - mean_absolute_error_labels: 26.6000 - val_loss: 48.6781 - val_mean_absolute_error_labels: 26.2632 - lr: 0.0010
Epoch 2/100
868/868 [==============================] - 172s 198ms/step - loss: 40.6281 - mean_absolute_error_labels: 14.5739 - val_loss: 37.1916 - val_mean_absolute_error_labels: 13.4859 - lr: 0.0010
Epoch 3/100
868/868 [==============================] - 172s 198ms/step - loss: 33.6719 - mean_absolute_error_labels: 11.5920 - val_loss: 32.7284 - val_mean_absolute_error_labels: 11.8056 - lr: 0.0010
Epoch 4/100
868/868 [==============================] - 173s 199ms/step - loss: 29.5450 - mean_absolute_error_labels: 10.3964 - val_loss: 30.2851 - val_mean_absolute_error_labels: 11.5794 - lr: 0.0010
Epoch 5/100
868/868 [==============================] - 172s 198ms/step - loss: 26.5988 - mean_absolute_error_labels: 9.5908 - val_loss: 28.6654 - val_mean_absolute_error_labels: 11.6457 - lr: 0.0010
E

In [7]:
model.save("D:/models/FINAL/Mobile/testing/IMDB.h5")
model_train_loss = score.history["loss"]
model_val_loss = score.history["val_loss"]
model_train_acc = score.history["mean_absolute_error_labels"]
model_val_acc = score.history["val_mean_absolute_error_labels"]

results = {'train-loss': model_train_loss,
        'val-loss': model_val_loss,
        'train-mae': model_train_acc,
        'val-mae': model_val_acc}
resultsdf = pd.DataFrame(results)
resultsdf.to_csv('D:/models/FINAL/Mobile/testing/IMDB.csv', index=False)
